In [1]:
import cv2
import mediapipe as mp
import numpy as np
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Define colors for drawing
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
color_index = 0

# Start video capture
cap = cv2.VideoCapture(0)

# Create a white canvas (will resize later)
paint_window = None

# Function to detect specific hand gestures
def detect_hand_gesture(image):
    results = hands.process(image)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            index_tip = (hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                         hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y)
            middle_tip = (hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,
                          hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y)
            # Calculate the distance between index and middle finger tips
            tip_distance = np.linalg.norm(np.array(index_tip) - np.array(middle_tip))
            if tip_distance < 0.05:
                return True
    return False

# Main loop for processing video frames
drawing_active = True
prev_cx, prev_cy = None, None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    if paint_window is None:
        # Create the paint window with the same size as the frame
        paint_window = np.ones_like(frame) * 255

    # Draw the UI buttons
    frame = cv2.rectangle(frame, (40, 1), (140, 65), (122, 122, 122), -1)
    frame = cv2.rectangle(frame, (160, 1), (255, 65), colors[0], -1)
    frame = cv2.rectangle(frame, (275, 1), (370, 65), colors[1], -1)
    frame = cv2.rectangle(frame, (390, 1), (485, 65), colors[2], -1)
    frame = cv2.rectangle(frame, (505, 1), (600, 65), colors[3], -1)

    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 150, 150), 2, cv2.LINE_AA)

    # Process hand landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            h, w, _ = frame.shape
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            # Debug: Draw a circle on the index finger tip
            cv2.circle(frame, (cx, cy), 10, (0, 0, 255), -1)

            # Check for UI button interaction
            if cy <= 65:
                if 40 <= cx <= 140:
                    paint_window[:] = 255  # Clear the canvas
                elif 160 <= cx <= 255:
                    color_index = 0
                elif 275 <= cx <= 370:
                    color_index = 1
                elif 390 <= cx <= 485:
                    color_index = 2
                elif 505 <= cx <= 600:
                    color_index = 3
            else:
                # Draw on the canvas only if the index finger is active
                if drawing_active:
                    if prev_cx is not None and prev_cy is not None:
                        # Draw a line between the previous and current points
                        cv2.line(paint_window, (prev_cx, prev_cy), (cx, cy), colors[color_index], 5)
                    # Update the previous position
                    prev_cx, prev_cy = cx, cy
                else:
                    # Reset the previous position if not drawing
                    prev_cx, prev_cy = None, None

    # Detect hand gesture to stop drawing
    if detect_hand_gesture(frame_rgb):
        drawing_active = False
    else:
        drawing_active = True

    # Overlay the paint window onto the frame
    combined_image = cv2.addWeighted(frame, 0.5, paint_window, 0.5, 0)

    cv2.imshow("Virtual Painter", combined_image)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 